In [19]:
import numpy as np
import tifffile
import matplotlib.pyplot as plt
import os
import glob
import keras
from keras.utils import np_utils
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam

In [20]:
folder = "D:\BScope\\SequentialBeads\\stacks\\TrainingData\\MaxProjection\\"

In [25]:
def onehot_encode(arr):
    
    uniques, ids = np.unique(arr, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))

In [27]:
# Label encoding
classes = os.listdir(folder)
onehot = onehot_encode(classes)

for i,c in enumerate(classes):
    print('%10s → [%d,%d]' % (c, onehot[i,0], onehot[i,1]))

ObliqueAstigmationMaxProjection → [1,0]
VerticalAstigmationMaxProjection → [0,1]


In [28]:
# Loading DataSet
imageData = list();
modes = list();
subDir = os.listdir(folder)

for s in subDir:
    print(folder+ s +"\\*.tif")
    files  = glob.glob(folder+ s +"\\*.tif" )
    for f in files:
        imageData.append(tifffile.imread(f))
        modes.append(s)

D:\BScope\SequentialBeads\stacks\TrainingData\MaxProjection\ObliqueAstigmationMaxProjection\*.tif
D:\BScope\SequentialBeads\stacks\TrainingData\MaxProjection\VerticalAstigmationMaxProjection\*.tif


In [29]:
# Normalization of the entrire stack
def entireStackNormalization(stackImgData): 
    # Mean and std of entire stack
    mean = np.mean(stackImgData)
    std = np.std(stackImgData)
    normalizedStackImgData = (stackImgData - mean) / std
    return normalizedStackImgData

In [30]:
# Normalization plane by plane
def planeByPlaneStackNormalization(stackImgData):
    # careful
    mean = np.mean(stackImgData, axis = 0)
    std = np.std(stackImgData, axis = 0)
    normalizedStackImgData = np.array([(stackImgData[i] - mean[i]) / std[i] for i in range(0,np.shape(stackImgData)[0])])
    return normalizedStackImgData

In [31]:
X = np.array([entireStackNormalization(i) for i in imageData])
modes = np.array(modes)

In [32]:
X = np.array(imageData)
Y = onehot_encode(modes)

In [33]:
print(X.shape,Y.shape)

(132, 64, 64) (132, 2)


In [34]:
# data size for conv layer
X_train = np.expand_dims(X,-1)
print(X_train.shape,Y.shape)

(132, 64, 64, 1) (132, 2)


In [38]:
# CNN Model
def get_model2():
    inp = Input(       name='X',     shape=X_train.shape[1:])    
    t   = Conv2D(32,   name='conv1', kernel_size=(3,3), activation='relu')(inp)
    t   = Conv2D(32,   name='conv2', kernel_size=(3,3), activation='relu')(t)
    t   = MaxPooling2D(name='pool1', pool_size=(2,2))(t)
    #t   = Conv2D(32,   name='conv3', kernel_size=(3,3), activation='relu')(t)
    t   = Conv2D(32,   name='conv4', kernel_size=(3,3), activation='relu')(t)
    t   = MaxPooling2D(name='pool2', pool_size=(2,2))(t)
    t   = Flatten(     name='flat')(t)
    t   = Dense(64,   name='dense', activation='relu')(t)
    oup = Dense(Y.shape[1],    name='Y', activation='softmax')(t)
    model = Model(inp,oup)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

In [39]:
model = get_model2()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X (InputLayer)               (None, 64, 64, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 62, 62, 32)        320       
_________________________________________________________________
conv2 (Conv2D)               (None, 60, 60, 32)        9248      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 30, 30, 32)        0         
_________________________________________________________________
conv4 (Conv2D)               (None, 28, 28, 32)        9248      
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 14, 14, 32)        0         
_________________________________________________________________
flat (Flatten)               (None, 6272)              0         
__________

In [ ]:
hist = model.fit(X_train,Y, batch_size=128, epochs=15,validation_split=0.1)

Train on 118 samples, validate on 14 samples
Epoch 1/15
118/118 [==============================] - 16s 136ms/step - loss: 7.1095 - acc: 0.4407 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/15
118/118 [==============================] - 15s 130ms/step - loss: 7.1029 - acc: 0.5593 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 3/15
118/118 [==============================] - 16s 134ms/step - loss: 7.1029 - acc: 0.5593 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 4/15
118/118 [==============================] - 16s 137ms/step - loss: 7.1029 - acc: 0.5593 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 5/15
118/118 [==============================] - 15s 129ms/step - loss: 7.1029 - acc: 0.5593 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 6/15
118/118 [==============================] - 15s 130ms/step - loss: 7.1029 - acc: 0.5593 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 7/15
118/118 [==============================] - 15s 129ms/step - loss: 7.1029 - acc: 0.5593 - val_loss: 16.11